## YOLO vs DENSENET
Tämä projekti vertailee YOLO-objektintunnistusta ja DenseNet-kuvaluokitusta materialdatasetillä:
- YOLO tunnistaa materialobjektit kuvista.
- DenseNet luokittelee tarkemmin minkälainen material on kyseessä.
- Tuloksista piirretään kuvat, joissa näkyy kummankin mallin ennusteet ja "hybridi"-tulos, joka yhdistää molemmat parhaalla tavalla.

## Vaihe 1: Kirjastojen ja työkalujen lataus

In [ ]:
import glob
import random
import os
import torch
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import models, transforms, datasets
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import logging
import yaml
from ultralytics import YOLO
from ultralytics.utils import LOGGER

In [ ]:
%matplotlib inline
LOGGER.setLevel(logging.ERROR)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Using device: {device}")

## Vaihe 2: YOLO mallin lataaminen
- Valmiiksi koulutettu materialihin liittyvä objektiluokitusmalli haetaan polusta. 

In [ ]:
# Lataa YOLO
yolo_model = YOLO('material_koulutus/train_reetta_v3/weights/best.pt') # Päivitä tämä omaan parhaaseen YOLO versioon

## Vaihe 3: DenseNet
DenseNet121 rakennetaan:
- Kuvakoko normalisoidaan (224 × 224 pikseliä).
- Mallin viimeinen luokituskerros (classifier) korvataan sopimaan projektin omiin luokkiin (class_names_from_folder).
- Mallin painot ladataan tiedostosta 'densenet_materials_best.pth'.
- Malli asetetaan evaluointitilaan (eval()).

In [ ]:
# Lataa DenseNet
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Huom Densenet ottaa vain tämän kokoisia kuvia, ei voi muuttaa
    transforms.ToTensor()
])

sample_dataset = datasets.ImageFolder('datasets/materials/cropped/train', transform=transform)
class_names_from_folder = sample_dataset.classes

densenet_model = models.densenet121(weights=None)
densenet_model.classifier = nn.Linear(densenet_model.classifier.in_features, len(class_names_from_folder))
densenet_model.load_state_dict(torch.load('densenet_materials_best.pth', map_location=device))
densenet_model.to(device)
densenet_model.eval()

## Vaihe 4: Vertailu
Tämä funktio:
- Lataa yksittäisen kuvan polusta.
- Suorittaa objektin tunnistuksen YOLO-mallilla.

Jokaiselle YOLO:n löytämälle alueelle:
- Rajaa kuvan osa.
- Syöttää raja-alueen DenseNet-mallille luokitusta varten.

Piirtää kolme eri versiota kuvasta:
- YOLO-luokitustulokset
- DenseNet-luokitustulokset
- Hybridimalli, joka valitsee YOLO:n tuloksen, jos sen varmuus (confidence) on >70 %, muuten DenseNetin tuloksen.

In [ ]:
# --- Näytä YOLO ja DenseNet tulokset ---
def show_yolo_vs_densenet(image_path, yolo_model, densenet_model, class_names):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Kuvaa ei löydy: {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    yolo_img, densenet_img, hybrid_img = img.copy(), img.copy(), img.copy()

    results = yolo_model(image_path)[0]
    font = cv2.FONT_HERSHEY_SIMPLEX

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        yolo_cls = int(box.cls[0])
        yolo_conf = float(box.conf[0])

        # DenseNet crop
        crop = densenet_img[y1:y2, x1:x2]
        if crop.size == 0:
            continue
        pil_crop = Image.fromarray(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        input_tensor = transform(pil_crop).unsqueeze(0).to(device)

        with torch.no_grad():
            output = densenet_model(input_tensor)
            pred_cls = output.argmax(dim=1).item()
            dn_conf = torch.softmax(output, dim=1)[0][pred_cls].item()

        # Piirrä YOLO ja DenseNet
        for target_img, label, color in [
            (yolo_img, f"{class_names[yolo_cls]} ({yolo_conf*100:.1f}%)", (0,0,0)),
            (densenet_img, f"{class_names[pred_cls]} ({dn_conf*100:.1f}%)", (0,0,0)),
        ]:
            cv2.rectangle(target_img, (x1, y1), (x2, y2), color, 3)
            (tw, th), _ = cv2.getTextSize(label, font, 1.2, 2)
            cv2.putText(target_img, label, (x1, max(y1-10, th)), font, 1.2, color, 2)

        # Hybrid
        if yolo_conf >= 0.7:
            final_cls, final_conf, source = yolo_cls, yolo_conf, "YOLO"
            color = (0, 0, 0)
        else:
            final_cls, final_conf, source = pred_cls, dn_conf, "DenseNet"
            color = (0, 0 ,0)

        label = f"{class_names[final_cls]} ({final_conf*100:.1f}%) [{source}]"
        cv2.rectangle(hybrid_img, (x1, y1), (x2, y2), color, 2)
        (tw, th), _ = cv2.getTextSize(label, font, 1.2, 2)
        cv2.putText(hybrid_img, label, (x1, max(y1-10, th)), font, 1.2, color, 2)

    # Näytä kuvat
    plt.figure(figsize=(18,6))
    for idx, (img, title) in enumerate(zip([yolo_img, densenet_img, hybrid_img],
                                           ["YOLO-luokitus", "DenseNet-luokitus", "Hybridiluokitus"])):
        plt.subplot(1,3,idx+1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title, fontsize=18)
        plt.axis("off")
    plt.tight_layout()
    plt.show()


## Vaihe 5: Visualisointi
- Haetaan kaikki validointikuvat kansiosta datasets/materials/images/validation/
- Valitaan satunnaisesti kolme kuvaa.
- Näytetään kunkin kuvan YOLO-, DenseNet- ja hybridiluokitus tulokset vierekkäin.

In [ ]:
# --- Arvo ja näytä 3 satunnaista validointikuvaa ---
image_files = glob.glob('datasets/materials/images/validation/*.jpg') + glob.glob('datasets/materials/images/validation/*.png')
random_images = random.sample(image_files, 3)

for path in random_images:
    show_yolo_vs_densenet(path, yolo_model, densenet_model, class_names_from_folder)

In [ ]:
# --- Lue YAML ja tulosta luokat ---
yaml_path = 'config/materials.yaml'

with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

names_dict = config['names']

# Tulosta siististi
print(' '.join(f"{idx}: {name}" for idx, name in names_dict.items()))

In [ ]:
print("DenseNet classes:", class_names_from_folder)
print("Number of classes:", len(class_names_from_folder))


## Vaihe 6: DenseNet evaluointi

In [ ]:
# --- Lataa validointidata ---
val_data = datasets.ImageFolder('datasets/materials/cropped/validation', transform=transform)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)

In [ ]:
# --- Evaluoi DenseNet ---
def evaluate_densenet(densenet_model, dataloader, device, class_names):
    densenet_model.eval()
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = densenet_model(images)
            preds = outputs.argmax(dim=1)

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    # Muutetaan names_dict listaksi
    class_names_list = [name for idx, name in sorted(names_dict.items())]

    print("\n DenseNetin tarkkuusluvut (validation-datalla):")
    print(classification_report(
        true_labels, 
        predicted_labels, 
        labels=list(range(len(class_names_list))), 
        target_names=class_names_list
    ))


In [ ]:
# --- Suorita evaluointi ---
evaluate_densenet(densenet_model, val_loader, device, names_dict)